In [1]:
import numpy as np
import struct
import pandas as pd
from collections import Counter
import os
from datetime import datetime, timedelta
import math
from scipy.fft import fft, fftfreq

In [2]:
def read_dbs_file(filepath):
    data = {
        'parameters': [], 'spectra': [], 'errors': [], 'beam_info': [], 'record_types': []
    }
    record_count = 0
    error_count = 0
    outlier_count = 0

    print(f"Reading Davis MST radar data from: {filepath}")
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"File {filepath} does not exist.")

    with open(filepath, 'rb') as f:
        while True:
            record = f.read(64)
            if not record:
                break
            record_count += 1
            if len(record) != 64:
                data['errors'].append(f"Record {record_count}: Incomplete record, {len(record)} bytes")
                error_count += 1
                continue

            if record_count <= 5:
                print(f"\nRecord {record_count} analysis:")
                print(f"Header bytes [0-1]: {hex(record[0])}, {hex(record[1])}")
                print(f"Float at [8-11]: {struct.unpack('>f', record[8:12])[0]:.6f}")
                print(f"Float at [12-15]: {struct.unpack('>f', record[12:16])[0]:.6f}")
                print(f"Bytes [16-19]: {[hex(b) for b in record[16:20]]}")
                print(f"Bytes [20-23]: {[hex(b) for b in record[20:24]]}")

            try:
                header = struct.unpack('>H', record[0:2])[0]
                param1 = struct.unpack('>f', record[8:12])[0]
                param2 = struct.unpack('>f', record[12:16])[0]
                aux_data = struct.unpack('>I', record[20:24])[0]
                beam_code = record[16] & 0x0F
                spectra = struct.unpack('>10f', record[24:64])

                if abs(param2) > 1e6:
                    data['errors'].append(f"Record {record_count}: Large param2 value: {param2:.2e}")
                    error_count += 1
                    outlier_count += 1
                    continue
                if any(abs(x) > 1e6 for x in spectra):
                    data['errors'].append(f"Record {record_count}: Large spectral value: {max(spectra, key=abs):.2e}")
                    error_count += 1
                    continue

                data['parameters'].append({'param1': param1, 'param2': param2, 'aux_data': aux_data})
                data['spectra'].append(spectra)
                data['beam_info'].append(beam_code)
                data['record_types'].append(header)

                if record_count <= 5:
                    print(f"First 3 spectral values: {spectra[:3]}")

            except struct.error as e:
                data['errors'].append(f"Record {record_count}: Parsing error: {e}")
                error_count += 1
                continue

    print(f"\nProcessing complete:")
    print(f"Total records: {record_count}")
    print(f"Successful records: {len(data['spectra'])}")
    print(f"Errors: {error_count}")
    print(f"Outlier param2 records: {outlier_count}")
    print(f"Sample errors: {data['errors'][:3]}")
    print(f"Record types: {[hex(t) for t in sorted(set(data['record_types']))]}")
    print(f"Beam codes: {Counter(data['beam_info'])}")

    return data

In [3]:
def analyze_spectra(data):
    if not data['spectra']:
        print("No spectral data to analyze")
        return None, None

    valid_indices = [i for i, s in enumerate(data['spectra']) if max(abs(np.array(s))) < 1e6]
    spectra_array = np.array([data['spectra'][i] for i in valid_indices])
    params_array = np.array([[data['parameters'][i]['param1'], data['parameters'][i]['param2'], data['parameters'][i]['aux_data']] for i in valid_indices])

    print(f"\nSpectral Data Analysis:")
    print(f"Shape: {spectra_array.shape}")
    print(f"Spectral range: {np.min(spectra_array):.3f} to {np.max(spectra_array):.3f}")
    print(f"Mean spectral value: {np.mean(spectra_array):.3f}")
    print(f"Std deviation: {np.std(spectra_array):.3f}")
    print(f"5th-95th percentile range: {np.percentile(spectra_array, 5):.3f} to {np.percentile(spectra_array, 95):.3f}")

    print(f"\nParameter Analysis:")
    print(f"Param1 (velocity?) range: {np.min(params_array[:, 0]):.6f} to {np.max(params_array[:, 0]):.6f}")
    print(f"Param1 mean: {np.mean(params_array[:, 0]):.6f}")
    print(f"Param2 (SNR?) range: {np.min(params_array[:, 1]):.6f} to {np.max(params_array[:, 1]):.6f}")
    print(f"Param2 mean: {np.mean(params_array[:, 1]):.6f}")

    print(f"\nBeam Analysis:")
    for beam in sorted(set(data['beam_info'])):
        count = sum(1 for i, b in enumerate(data['beam_info']) if b == beam and i in valid_indices)
        print(f"  Beam {beam}: {count} records ({count/len(valid_indices)*100:.1f}%)")

    # Wind Calculations
    velocity_bins = np.linspace(-5, 5, 10)
    beam_angles = {0: 0, 1: 15, 2: 15, 13: 15, 14: 15, 15: 15}
    wind_components = []
    for i in valid_indices:
        spec = data['spectra'][i]
        beam = data['beam_info'][i]
        vr = velocity_bins[np.argmax(spec)]
        theta = math.radians(beam_angles.get(beam, 15))
        if beam == 0:
            wind_components.append((0, 0, vr))
        elif beam == 1:
            wind_components.append((0, vr / math.sin(theta), 0))
        elif beam == 2:
            wind_components.append((vr / math.sin(theta), 0, 0))
        elif beam == 14:
            wind_components.append((-vr / math.sin(theta), 0, 0))
        elif beam == 13:
            wind_components.append((0, -vr / math.sin(theta), 0))
        elif beam == 15:
            wind_components.append((0, 0, vr))
    if wind_components:
        u, v, w = np.mean(wind_components, axis=0)
        u_std, v_std, w_std = np.std(wind_components, axis=0)
        print(f"\nWind Analysis:")
        print(f"  u={u:.2f} m/s (eastward), v={v:.2f} m/s (northward), w={w:.2f} m/s (vertical)")
        print(f"  Uncertainty: u_std={u_std:.2f} m/s, v_std={v_std:.2f} m/s, w_std={w_std:.2f} m/s")

        # SNR-weighted winds
        weights = [data['parameters'][i]['param2'] if data['parameters'][i]['param2'] > 0 else 1e-6 for i in valid_indices]
        weights = np.array(weights) / np.sum(weights)
        u_w, v_w, w_w = np.average(wind_components, axis=0, weights=weights)
        print(f"  Weighted wind: u={u_w:.2f} m/s, v={v_w:.2f} m/s, w={w_w:.2f} m/s")

        # SNR-filtered winds
        snr_threshold = 0
        valid_wind_indices = [i for i in valid_indices if data['parameters'][i]['param2'] > snr_threshold]
        wind_components_snr = []
        for i in valid_wind_indices:
            spec = data['spectra'][i]
            beam = data['beam_info'][i]
            vr = velocity_bins[np.argmax(spec)]
            theta = math.radians(beam_angles.get(beam, 15))
            if beam == 0:
                wind_components_snr.append((0, 0, vr))
            elif beam == 1:
                wind_components_snr.append((0, vr / math.sin(theta), 0))
            elif beam == 2:
                wind_components_snr.append((vr / math.sin(theta), 0, 0))
            elif beam == 14:
                wind_components_snr.append((-vr / math.sin(theta), 0, 0))
            elif beam == 13:
                wind_components_snr.append((0, -vr / math.sin(theta), 0))
            elif beam == 15:
                wind_components_snr.append((0, 0, vr))
        if wind_components_snr:
            u_snr, v_snr, w_snr = np.mean(wind_components_snr, axis=0)
            u_snr_std, v_snr_std, w_snr_std = np.std(wind_components_snr, axis=0)
            print(f"  SNR-filtered wind: u={u_snr:.2f} m/s, v={v_snr:.2f} m/s, w={w_snr:.2f} m/s")
            print(f"  Uncertainty: u_std={u_snr_std:.2f} m/s, v_std={v_snr_std:.2f} m/s, w_std={w_snr_std:.2f} m/s")

    # Time-Resolved Winds
    start_time = datetime(2003, 10, 29)
    cycle_time = 180  # Seconds (3 minutes)
    timestamps = [start_time + timedelta(seconds=i * cycle_time) for i in range(len(data['record_types']))]
    time_bins = pd.date_range(start=start_time, end=start_time + timedelta(hours=48), freq='h')
    wind_time_series = []
    for i in range(len(time_bins)-1):
        indices = [j for j, t in enumerate(timestamps) if time_bins[i] <= t < time_bins[i+1] and j in valid_indices]
        if not indices:
            continue
        wind_components = []
        for idx in indices:
            spec = data['spectra'][idx]
            beam = data['beam_info'][idx]
            vr = velocity_bins[np.argmax(spec)]
            theta = math.radians(beam_angles.get(beam, 15))
            if beam == 0:
                wind_components.append((0, 0, vr))
            elif beam == 1:
                wind_components.append((0, vr / math.sin(theta), 0))
            elif beam == 2:
                wind_components.append((vr / math.sin(theta), 0, 0))
            elif beam == 14:
                wind_components.append((-vr / math.sin(theta), 0, 0))
            elif beam == 13:
                wind_components.append((0, -vr / math.sin(theta), 0))
            elif beam == 15:
                wind_components.append((0, 0, vr))
        if wind_components:
            u, v, w = np.mean(wind_components, axis=0)
            wind_time_series.append((time_bins[i], u, v, w))
    if wind_time_series:
        print("\nHourly Wind Analysis:")
        for t, u, v, w in wind_time_series:
            print(f"{t}: u={u:.2f} m/s, v={v:.2f} m/s, w={w:.2f} m/s")

    # Tidal and Gravity Wave Analysis
    if wind_time_series:
        times = [t for t, _, _, _ in wind_time_series]
        u_values = [u for _, u, _, _ in wind_time_series]
        v_values = [v for _, _, v, _ in wind_time_series]
        w_values = [w for _, _, _, w in wind_time_series]
        time_hours = [(t - times[0]).total_seconds() / 3600 for t in times]
        interp_times = np.linspace(0, max(time_hours), len(time_hours))
        u_interp = np.interp(interp_times, time_hours, u_values)
        v_interp = np.interp(interp_times, time_hours, v_values)
        w_interp = np.interp(interp_times, time_hours, w_values)
        
        N = len(u_interp)
        T = 3600  # 1 hour in seconds
        xf = fftfreq(N, T)[:N//2]
        periods = 1/xf[1:] / 3600 if len(xf) > 1 else []
        
        for component, label in [(u_interp, 'u'), (v_interp, 'v'), (w_interp, 'w')]:
            yf = fft(component)
            amplitudes = 2.0/N * np.abs(yf[0:N//2])
            print(f"\nTidal Analysis ({label}-component):")
            for period, amp in zip(periods, amplitudes[1:]):
                if 10 < period < 30:
                    print(f"Period: {period:.1f} hours, Amplitude: {amp:.2f} m/s")
            print(f"\nGravity Wave Analysis ({label}-component):")
            component_detrended = component - np.mean(component)
            yf = fft(component_detrended)
            amplitudes = 2.0/N * np.abs(yf[0:N//2])
            for period, amp in zip(periods, amplitudes[1:]):
                if 1 < period < 6:
                    frequency = 1 / (period * 3600)  # Hz
                    phase_speed = 50e3 * frequency  # Assuming 50 km wavelength
                    wavelength = phase_speed / frequency / 1e3  # km
                    print(f"Period: {period:.1f} h, Amplitude: {amp:.2f} m/s, Phase Speed: {phase_speed:.2f} m/s, Wavelength: {wavelength:.1f} km")

    return spectra_array, params_array

In [4]:
def export_data(data, output_prefix='davis_mst'):
    if not data['spectra']:
        print("No data to export")
        return

    valid_indices = [i for i, s in enumerate(data['spectra']) if max(abs(np.array(s))) < 1e6]
    spectra_array = np.array([data['spectra'][i] for i in valid_indices])
    params_array = np.array([[data['parameters'][i]['param1'], data['parameters'][i]['param2'], data['parameters'][i]['aux_data']] for i in valid_indices])
    beam_array = np.array([data['beam_info'][i] for i in valid_indices])
    np.savez(f'{output_prefix}_data.npz', spectra=spectra_array, parameters=params_array, beams=beam_array)

    start_time = datetime(2003, 10, 29)
    cycle_time = 180
    timestamps = [start_time + timedelta(seconds=i * cycle_time) for i in range(len(data['record_types']))]
    df_params = pd.DataFrame([data['parameters'][i] for i in valid_indices])
    df_params['beam_code'] = [data['beam_info'][i] for i in valid_indices]
    df_params['record_type'] = [hex(data['record_types'][i]) for i in valid_indices]
    df_params['timestamp'] = [timestamps[i] for i in valid_indices]
    df_spectra = pd.DataFrame(spectra_array, columns=[f"bin_{i+1}" for i in range(10)])
    df_params.to_csv(f'{output_prefix}_params.csv', index=False)
    df_spectra.to_csv(f'{output_prefix}_spectra.csv', index=False)

    print(f"\nData exported to {output_prefix}_data.npz, {output_prefix}_params.csv, and {output_prefix}_spectra.csv")

In [5]:
# Executing the code
try:
    filepath = 'Davis_MST_Radar/DBS_meso/20031029_meso.dbs'
    data = read_dbs_file(filepath)
    if data['spectra']:
        print("\n" + "="*50)
        print("SUCCESSFULLY PARSED")
        print("="*50)
        spectra_array, params_array = analyze_spectra(data)
        # export_data(data, 'davis_mst_20031029')
except FileNotFoundError:
    print("Error: File not found.")
except Exception as e:
    print(f"Unexpected error: {e}")

Reading Davis MST radar data from: Davis_MST_Radar/DBS_meso/20031029_meso.dbs

Record 1 analysis:
Header bytes [0-1]: 0x20, 0x0
Float at [8-11]: 1.244854
Float at [12-15]: 1.252150
Bytes [16-19]: ['0x20', '0x30', '0x0', '0x0']
Bytes [20-23]: ['0x0', '0x0', '0x0', '0x24']

Record 2 analysis:
Header bytes [0-1]: 0x41, 0x5a
Float at [8-11]: 0.000000
Float at [12-15]: 1.244854
Bytes [16-19]: ['0x0', '0x0', '0x0', '0x54']
Bytes [20-23]: ['0x0', '0x0', '0x0', '0x0']
First 3 spectral values: (-4.044689178466797, 0.08582928776741028, -20.452547073364258)

Record 3 analysis:
Header bytes [0-1]: 0x41, 0x1a
Float at [8-11]: -16.699446
Float at [12-15]: 46.586494
Bytes [16-19]: ['0x0', '0x0', '0x0', '0x0']
Bytes [20-23]: ['0x41', '0x21', '0x75', '0xc8']
First 3 spectral values: (0.2909150719642639, -15.59093952178955, 47.74790954589844)

Record 4 analysis:
Header bytes [0-1]: 0x3e, 0xd1
Float at [8-11]: 46.697701
Float at [12-15]: 0.000000
Bytes [16-19]: ['0x41', '0x1f', '0xfd', '0xff']
Bytes [20-